# (Singular Value Thresholding, SVT) model

### Singular Value Thresholding (SVT) is an algorithm to minimize the nuclear norm of a matrix, subject to certain types of constraints. It has been successfully used in many matrix-completion problems. 

### Actually, the essence of this kaggle competition is to fill missing values in the sequence. So, this method can be applied to this scenario.

## 1. Defination of SVT model

In [1]:
import numpy as np
from numpy.linalg import inv
import pandas as pd

In [2]:
def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

In [3]:
def mat2ten(mat, tensor_size, mode):
    index = list()
    index.append(mode)
    for i in range(tensor_size.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, list(tensor_size[index]), order = 'F'), 0, mode)

In [4]:
def svt_tnn(mat, alpha, rho, theta):
    """This is a Numpy dependent singular value thresholding (SVT) process."""
    u, s, v = np.linalg.svd(mat, full_matrices = 0)
    vec = s.copy()
    vec[theta :] = s[theta :] - alpha / rho
    vec[vec < 0] = 0
    return np.matmul(np.matmul(u, np.diag(vec)), v)

In [5]:
def compute_mse(var, var_hat):
    return np.sum((var - var_hat) ** 2) / var.shape[0]

In [6]:
def LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter):
    """Low-Rank Tenor Completion with Truncated Nuclear Norm, LRTC-TNN."""
    
    dim = np.array(sparse_tensor.shape)
    pos_missing = np.where(sparse_tensor == 0)
    pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0)) #Missed marked data
    
    X = np.zeros(np.insert(dim, 0, len(dim))) # \boldsymbol{\mathcal{X}}
    T = np.zeros(np.insert(dim, 0, len(dim))) # \boldsymbol{\mathcal{T}}
    Z = sparse_tensor.copy()
    last_tensor = sparse_tensor.copy()
    snorm = np.sqrt(np.sum(sparse_tensor ** 2))
    it = 0
    while True:
        rho = min(rho * 1.05, 1e5)
        for k in range(len(dim)):
            X[k] = mat2ten(svt_tnn(ten2mat(Z - T[k] / rho, k), alpha[k], rho, np.int(np.ceil(theta * dim[k]))), dim, k)
        Z[pos_missing] = np.mean(X + T / rho, axis = 0)[pos_missing]
        T = T + rho * (X - np.broadcast_to(Z, np.insert(dim, 0, len(dim))))
        tensor_hat = np.einsum('k, kmnt -> mnt', alpha, X)
        tol = np.sqrt(np.sum((tensor_hat - last_tensor) ** 2)) / snorm
        last_tensor = tensor_hat.copy()
        it += 1
        if (it + 1) % 10 == 0:
            print('Iter: {}'.format(it + 1))
            print('MSE: {:.6}'.format(compute_mse(dense_tensor[pos_test], tensor_hat[pos_test])))
            print()
        if (tol < epsilon) or (it >= maxiter):
            break

    print('Imputation MSE: {:.6}'.format(compute_mse(dense_tensor[pos_test], tensor_hat[pos_test])))
    print()
    
    return tensor_hat

## 2. Data preparation for SVT model

In [7]:
train=pd.read_csv('./data/train.csv',index_col='id')
test=pd.read_csv('./data/test.csv',index_col='id')

In [8]:
all_data=pd.concat([train,test],ignore_index=True)
all_data.rename(columns={'date': 'datetime'},inplace=True)
all_data['datetime']=pd.to_datetime(all_data['datetime'],format='%d/%m/%Y %H:%M')
all_data = all_data.sort_values(by='datetime').reset_index(drop=True)

In [9]:
# we need to find the missing date and make the time of data complete
tmp=pd.Series(pd.date_range(start = '1/1/2017',periods = 24*365*2,freq = 'h'))
tmp=pd.DataFrame(tmp,columns=['datetime'])

all_data1=pd.merge(tmp,all_data,how='left')
all_data1.speed.fillna(0,inplace=True)

In [10]:
all_data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17520 entries, 0 to 17519
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   datetime  17520 non-null  datetime64[ns]
 1   speed     17520 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 410.6 KB


In [11]:
dense_tensor=np.array(all_data1.speed).reshape(1,730,24)

In [12]:
np.random.seed(101)
random_tensor=np.random.rand(1,730,24)

In [13]:
random_tensor.shape

(1, 730, 24)

## 3. Running of SVT model

In [14]:
missing_rate = 0.02

# =============================================================================
### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
# =============================================================================

sparse_tensor = np.multiply(dense_tensor, binary_tensor)

In [15]:
len(np.where(sparse_tensor==0)[0])

3809

In [16]:
import time
start = time.time()
alpha = np.ones(3) / 3
rho = 1e-5
theta = 0.31
epsilon = 1e-5
maxiter = 120

tensor_hat=LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 10
MSE: 80.5854

Iter: 20
MSE: 10.125

Iter: 30
MSE: 9.36305

Iter: 40
MSE: 10.1706

Iter: 50
MSE: 9.72531

Iter: 60
MSE: 9.29802

Iter: 70
MSE: 9.56929

Iter: 80
MSE: 9.81663

Iter: 90
MSE: 9.93628

Iter: 100
MSE: 9.35659

Iter: 110
MSE: 9.43214

Iter: 120
MSE: 9.30855

Imputation MSE: 9.02557

Running time: 0 seconds


## 4. Transfer the preditions to sample submission

In [17]:
# then we can get all the reconstructed data
tensor_hat=tensor_hat.reshape(-1)

In [18]:
tensor_hat.shape

(17520,)

In [19]:
svt_prediction = pd.DataFrame(tensor_hat)

In [20]:
# find the index of missing speed 
loss_index = list(all_data1[all_data1.speed==0].index)

svt_prediction = svt_prediction.iloc[loss_index,:]
# drop the value of missing date
svt_prediction=svt_prediction.iloc[10:,:]

In [21]:
svt_prediction = svt_prediction.reset_index(drop=True)
svt_prediction['id'] = svt_prediction.index
svt_prediction.columns = ['speed', 'id']
cols = ['id', 'speed']
svt_prediction = svt_prediction.loc[:,cols]
svt_prediction.to_csv('./svt_speed_prediction.csv',index=False)

In [22]:
svt_prediction

,id,speed
0,0,48.318035
1,1,47.923274
2,2,35.706701
3,3,30.041960
4,4,37.763385
...,...,...
3499,3499,14.042840
3500,3500,27.372527
3501,3501,41.701046
3502,3502,35.049542
